In [ ]:
import os
import cv2
import csv
import time
import psutil
import pyvisa
import threading
import numpy as np
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt

def grabFrame(cap, scaleFactor):
    ret, frame = cap.read(); 
    dim_l = int(frame.shape[1]*scaleFactor); dim_h = int(frame.shape[0]*scaleFactor); dim = (dim_l, dim_h) ## downscale
    raw = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA); ## resize
    out = raw.copy() ## backup
    
    return [raw, out, dim]

## params
resWidth = [1600, 1920, 2560, 3856]
resHeight = [1200, 1080, 1920, 2764]
resFPS = [120, 66.51, 29.86, 14.15]
option = 3; scaleFactor = 0.25

## open cam
cap1 = cv2.VideoCapture(0, cv2.CAP_DSHOW); ## first camera, top
if not cap1.isOpened(): raise IOError("cant open camera 1")
cap1.set(cv2.CAP_PROP_FRAME_WIDTH, resWidth[option]); cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, resHeight[option]); cap1.set(cv2.CAP_PROP_FPS, resFPS[option])

## main
while True:
    dataTop = grabFrame(cap1, scaleFactor); 
    
    raw = dataTop[0];  dim1 = dataTop[2]
    gscale = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY) ## convert to grayscaleFactor
    gaussian = cv2.GaussianBlur(gscale, (1,1), 0) ## gaussian blur

    cv2.imshow('raw', raw)
    cv2.imshow('gscale', gscale)
    cv2.imshow('gaussian', gaussian)
    
    pressedKey = cv2.waitKey(1) & 0xFF
    if pressedKey == ord('q'):
        cv2.destroyAllWindows()
        break
        
cap1.release()
cv2.destroyAllWindows()

In [ ]:
%matplotlib widget

import os
import cv2
import csv
import time
import psutil
import pyvisa
import threading
import numpy as np
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt

## params
resWidth = [1600, 1920, 2560, 3856]
resHeight = [1200, 1080, 1920, 2764]
resFPS = [120, 66.51, 29.86, 14.15]
option = 3; scaleFactor = 0.05

def grabFrame(cap, scaleFactor):
    ret, frame = cap.read(); 
    dim_l = int(frame.shape[1]*scaleFactor); dim_h = int(frame.shape[0]*scaleFactor); dim = (dim_l, dim_h) ## downscale
    raw = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA); ## resize
    out = raw.copy() ## backup
    
    return [raw, out, dim]

## open cam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW); ## first camera, top
if not cap.isOpened(): raise IOError("cant open camera 1")
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resWidth[option]); cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resHeight[option]); cap.set(cv2.CAP_PROP_FPS, resFPS[option])

## random data
data = np.random.randint(0, 50, size=(100, 100, 100))

fig, axes = plt.subplots(1, 1, layout='constrained')

# slider = widgets.IntSlider(value=0, min=0, max=data.shape[0]-1)
# play_button = widgets.Play(value=0, min=0, max=data.shape[0]-1,step=1,interval=200)
# widgets.jslink((play_button,"value"),(slider,"value"))

# def slider_update(change):
#     axes.imshow(data[change.new], cmap='jet', origin='lower', vmax=data.max(), vmin=data.min())
#     fig.canvas.draw_idle()
#     plt.suptitle(f'Time: {slider.value}')
# slider.observe(slider_update, 'value')

out = widgets.Output()
app = widgets.HBox([out, out, out])
display(app)

while True:
    dataTop = grabFrame(cap, scaleFactor)
    raw = dataTop[0]
    cv2.imshow('raw', raw)
    print('test')
    time.sleep(0.1)

    pressedKey = cv2.waitKey(1) & 0xFF
    if pressedKey == ord('q'):
        cv2.destroyAllWindows()
        break


In [ ]:
%matplotlib widget

import os
import cv2
import csv
import time
import psutil
import pyvisa
import threading
import numpy as np
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
import matplotlib.pyplot as plt

plt.rcParams["figure.autolayout"] = True; plt.rc('font', size=16)
plt.rc('text', usetex=False); plt.rc('font', family='serif'); plt.rcParams["figure.figsize"] = (8, 5)

# init params
output = widgets.Output()
cpuVal = -300

resWidth = [1600, 1920, 2560, 3856]
resHeight = [1200, 1080, 1920, 2764]
resFPS = [120, 66.51, 29.86, 14.15]
option = 3; scaleFactor = 0.5

cap = []

## outside accordion
cpuUsageBox = widgets.Text(description='CPU Usage :')

## first accordion 
plotSweepButton = widgets.Button(description='Plot')
plotSweepBox = widgets.Output()
templateHandler = widgets.VBox([plotSweepButton, plotSweepBox])

## second accordion 
plotTestButton = widgets.ToggleButton(description='Start Stream', calue=False)
plotTestBox = widgets.Output()
testHandler = widgets.VBox([plotTestButton, plotTestBox])

## all accordion
AccordHandler = widgets.Accordion(children=[templateHandler, testHandler])
AccordHandler.set_title(0, 'Template'); AccordHandler.set_title(1, 'Test')

## display widgets
display(
    widgets.VBox([widgets.Label('Template Widgets'),
        cpuUsageBox,
        AccordHandler]), 
    output)

def grabFrame(cap, scaleFactor):
    ret, frame = cap.read(); 
    dim_l = int(frame.shape[1]*scaleFactor); dim_h = int(frame.shape[0]*scaleFactor); dim = (dim_l, dim_h) ## downscale
    raw = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA); ## resize
    out = raw.copy() ## backup
    
    return [raw, out, dim]

def plotSweepButtonHandler(val):
    with plotSweepBox:
        plt.close('all')
        plotSweepBox.clear_output()
        figHandler, axHandler = plt.subplots(1, 1, layout='constrained')
        
        dataX = np.random.randn(100) * 2
        dataY = np.sin(dataX)

        plt.scatter(dataX, dataY)
        plt.grid(True); plt.xlabel('Wavelength [nm]'); plt.ylabel('Transmission [dBm]')
        plt.show()
plotSweepButton.on_click(plotSweepButtonHandler)

# def plotTestButtonHandler(val):
#     with plotTestBox:
#         if plotTestButton.value == True:
#             plt.close('all'); plotTestBox.clear_output()
#             figHandler, axHandler = plt.subplots(1, 1, layout='constrained')

#             while True:
#                 # dataTop = grabFrame(cap, scaleFactor)
#                 # img2 = dataTop[0][:,:,::-1]
#                 # plt.imshow(img2)
#                 # plt.grid(True); plt.xlabel('Wavelength [nm]'); plt.ylabel('Transmission [dBm]')
#                 # plt.show()

#                 dataTop = grabFrame(cap, scaleFactor)
#                 raw = dataTop[0]
#                 cv2.imshow('raw', raw)

#                 pressedKey = cv2.waitKey(1) & 0xFF
#                 if pressedKey == ord('q'):
#                     cv2.destroyAllWindows()
#                     break
#         else: 
#             plt.close('all'); plotTestBox.clear_output()
# plotTestButton.observe(plotTestButtonHandler, 'value')

## open cam
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW); ## first camera, top
if not cap.isOpened(): raise IOError('Cant open camera 1')
cap.set(cv2.CAP_PROP_FRAME_WIDTH, resWidth[option]); cap.set(cv2.CAP_PROP_FRAME_HEIGHT, resHeight[option]); cap.set(cv2.CAP_PROP_FPS, resFPS[option])

## readout function
def updateReadout():
    while True:
        
        cpuVal = psutil.cpu_percent(); cpuUsageBox.value = f'{float(cpuVal):0.3f}'

        time.sleep(1)

def updateCam():
    while True:
        plotStatus = plotTestButton.value
        if plotStatus == True: 
            while True:
                frame = grabFrame(cap, scaleFactor)
                
                raw = frame[0];  dim = frame[2]
                gscale = cv2.cvtColor(raw, cv2.COLOR_BGR2GRAY) ## convert to grayscaleFactor
                gaussian = cv2.GaussianBlur(gscale, (1,1), 0) ## gaussian blur

                cv2.imshow('raw', raw)
                cv2.imshow('gscale', gscale)
                cv2.imshow('gaussian', gaussian)
                
                cv2.imshow('test', cv2.hconcat([gscale, gaussian]))
                pressedKey = cv2.waitKey(1) & 0xFF
                if pressedKey == ord('q'):
                    cv2.destroyAllWindows()
                    plotTestButton.value = False
                    break

## threadings
threadsHandler = [] 
threadData = threading.Thread(target=updateReadout); threadsHandler.append(threadData)
threadData = threading.Thread(target=updateCam); threadsHandler.append(threadData)

for i in threadsHandler:
    i.start()